In [1]:
import os
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# import time for timekeeping
import time
import random
random.seed(1)

In [2]:
def remove_files_in_folder(folder_path):
    try:
        # List all files in the folder
        files = os.listdir(folder_path)

        # Loop through the files and remove them
        for file in files:
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Removed: {file_path}")

        #print("All files in the folder have been removed.")

    except Exception as e:
        print(f"An error occurred: {e}")
        
# Returns a transparent image given an img object in "L" format
def convert_to_transparent(img):
    img = img.convert("LA")

    datas = img.getdata()
    width, height = img.size

    new_img = Image.new("LA", (width, height))

    for i, item in enumerate(datas):
        if item[0] == 0:  # Check if greyscale value is 0
            new_img.putpixel((i % width, i // width), (0, 0))
        else:
            new_img.putpixel((i % width, i // width), item)
            
    return new_img
                        
    
def convertImages(number_of_output, input_fruit, input_label,input_background ,output_image,output_label):
    start = time.time()
    
    # create output folder if it doesn't exist already
    if not os.path.exists(output_image):
        os.makedirs(output_image)
    # else:
        # remove_files_in_folder(output_image)
    if not os.path.exists(output_label):
        os.makedirs(output_label)
    # else:
        # remove_files_in_folder(output_label)
        
    ########################################################################################   
    
    input_bg_list = os.listdir(input_background)
    input_fruit_list = os.listdir(input_fruit)
    input_label_list = os.listdir(input_label)
    
    if(len(input_fruit_list) != len(input_label_list)):
        print("fruit" + str(len(input_fruit_list)))
        print("label" + str(len(input_label_list)))
    
    ## for loop for number of pictures to generate   
    for pic_num in range(number_of_output):

        # boolean term for 20% chance of getting two fruits in the same image
        bool_get_two_fruit = random.randrange(10) >= 0
        bool_flip_image_horizontal = random.randrange(2)
        bool_flip_image_vertical = random.randrange(2)
        
        ## Generate random values to select pictures randomly
        # fruit & label shares the same index
        # print(len(input_fruit_list))
        index_fruit = random.randrange(1,len(input_fruit_list)-1)
        index_bg = random.randrange(1,len(input_bg_list)-1)

        # use the random value to pick from the list
        filename_bg = input_bg_list[index_bg]
        # print(index_fruit)
        filename_fruit = input_fruit_list[index_fruit]
        filename_label = input_label_list[index_fruit]
        
        # pick another fruit picture for double fruit
        if(bool_get_two_fruit == True):
            index_fruit = random.randrange(1,len(input_fruit_list)-1)
            # print(index_fruit)
            filename_fruit2 = input_fruit_list[index_fruit]
            filename_label2 = input_label_list[index_fruit]
            
            
    ######################################################################################## 
        ## open up the pictures
        if filename_bg.endswith(".png"):
            img_path_bg = os.path.join(input_background, filename_bg)
            img_bg = Image.open(img_path_bg)
            
            if (bool_flip_image_horizontal):
                img_bg = img_bg.transpose(Image.FLIP_LEFT_RIGHT)
                            
            if (bool_flip_image_vertical):
                img_bg = img_bg.transpose(Image.FLIP_TOP_BOTTOM)
            
            img_bg = img_bg.convert("RGBA")

            img_path_fruit = os.path.join(input_fruit, filename_fruit)
            img_fruit = Image.open(img_path_fruit)
            img_fruit = img_fruit.convert("RGBA")

            img_path_label = os.path.join(input_label, filename_label)
            img_label = Image.open(img_path_label)
            img_label = convert_to_transparent(img_label)
            
            # additional picture when there is a double fruit
            img_path_label2 = ""
            if(bool_get_two_fruit == True):
                img_path_fruit2 = os.path.join(input_fruit, filename_fruit2)
                img_fruit2 = Image.open(img_path_fruit2)
                img_fruit2 = img_fruit2.convert("RGBA")

                img_path_label2 = os.path.join(input_label, filename_label2)
                img_label2 = Image.open(img_path_label2)
                img_label2 = convert_to_transparent(img_label2)
            
            # obtain image size to match
            width, height = img_bg.size
            img_label_output = Image.new(mode="LA", size=(width, height))
            width  -= 80
            height -= 200
        ######################################################################################## 
            
            # set random values and coordinates
            scale = random.uniform(0.1, 1.2)
            ranx = random.randrange(-width/2,width/2)
            rany = random.randrange(-height/2,height/2)

            # apply random scaling to fruit image and label image
            img_fruit = img_fruit.resize((int(scale*640),int(scale*480)))
            img_label = img_label.resize((int(scale*640),int(scale*480)))

            # Pasting fruits using coordinate onto background, do the same for label on a blank background
            img_bg.paste(img_fruit, (ranx,rany), mask = img_fruit)
            
            
            img_label_output.paste(img_label, (ranx,rany), mask = img_label)
            
            
            if(bool_get_two_fruit == True):
                scale = random.uniform(0.1, 1.2)
                ranx = random.randrange(-width/2,width/2)
                rany = random.randrange(-height/2,height/2)
                
                img_fruit2 = img_fruit2.resize((int(scale*640),int(scale*480)))
                img_bg.paste(img_fruit2, (ranx,rany), mask = img_fruit2)
                
                img_label2 = img_label2.resize((int(scale*640),int(scale*480)))
                img_label_output.paste(img_label2, (ranx,rany), mask = img_label2)
                
                
            img_bg = img_bg.convert("RGB")
            img_label_output = img_label_output.convert("L")
            
            
            # Warning: This only works with 2 fruits
            if(bool_get_two_fruit == True):
                mpimg_label = mpimg.imread(img_path_label)
                mpimg_label2 = mpimg.imread(img_path_label2)
                img_label = img_label.convert("L")
                img_label2 = img_label2.convert("L")
            
                label_value_1, label_value_2 = mpimg_label.max()*255, mpimg_label2.max()*255
                overlap_value = label_value_1 + label_value_2

                # Find overlap order (assume overlapping fruit contains more pixel)
                datas_1, datas_2, datas_output = img_label.getdata(), img_label2.getdata(), img_label_output.getdata()
                pixel_count_1, pixel_count_2 = 0,0
                
                for a, item in enumerate(datas_1):
                    if item == label_value_1:
                        pixel_count_1+=1
                
                for b, item in enumerate(datas_2):
                    if item == label_value_2:
                        pixel_count_2+=1
                
                if pixel_count_1 > pixel_count_2:
                    priority_label = label_value_1

                # Replace overlapping pixels with priority pixel value
                width, height = img_label_output.size
                for i, item in enumerate(datas_output):
                    if item == overlap_value:
                        newImage.putpixel((i % width, i// width), priority_label)
                    

        ######################################################################################## 
            ## save the pictures into the directed folder
            extension = ".png"
            save_filename_fruit = os.path.join(output_image, "image_"  + str(pic_num+2000)+ extension)
            img_bg.save(save_filename_fruit, "PNG")

            save_filename_label = os.path.join(output_label, "image_"  + str(pic_num+2000) + "_label"+ extension)
            img_label_output.save(save_filename_label, "PNG")
            
            if((pic_num%100==0) or pic_num==(number_of_output-1)):
                end = time.time()
                print("Saved", save_filename_fruit,",  Saved", save_filename_label,",  time taken: " , round((end - start) / 60,3), "min") # time in minutes

    

In [3]:
start = time.time()

convertImages(number_of_output = 200, 
              input_fruit      = "./2_removedbg/sample_all/", 
              input_label      ="./3_label_pictures/sample_all/", #"./2_removedbg/apple/", #  
              input_background = "./4_arena_pics/" ,
              output_image     = "5_NN_Ready/images",
              output_label     = "5_NN_Ready/labels")

end = time.time()
print((end - start) / 60) # time in minutes

Saved 5_NN_Ready/images\image_2000.png ,  Saved 5_NN_Ready/labels\image_2000_label.png ,  time taken:  0.011 min
Saved 5_NN_Ready/images\image_2100.png ,  Saved 5_NN_Ready/labels\image_2100_label.png ,  time taken:  2.14 min
Saved 5_NN_Ready/images\image_2199.png ,  Saved 5_NN_Ready/labels\image_2199_label.png ,  time taken:  4.468 min
4.4679141680399574
